# VacationPy

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [25]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [31]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)
# Convert the temperature from Kelvin to Celsius
city_data_df['Max Temp'] = city_data_df['Max Temp'] - 273.15

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kodiak,57.7900,-152.4072,2.10,85,98,1.20,US,1704542009
1,1,zhangye,38.9342,100.4517,-2.50,35,6,1.91,CN,1704541712
2,2,touros,-5.1989,-35.4608,29.48,62,96,5.38,BR,1704542011
3,3,west island,-12.1568,96.8225,27.99,89,40,4.12,CC,1704542011
4,4,sorong,-0.8833,131.2500,27.66,78,100,1.16,ID,1704542013


# Step 1: Create a map that displays a point for every city in the city_data_df DataFrame. The size of the point should be the humidity in each city.

In [32]:

# Configure the map plot
map_plot_1= city_data_df.hvplot.points("Lng",
                                       "Lat",
                                       geo =True, 
                                       tiles = "OSM",
                                       frame_width=650,
                                       frame_height=500, 
                                       size="Humidity", 
                                       color="City"
                                      )

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

# Step 2: Narrow down the city_data_df DataFrame to find your ideal weather condition

In [33]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cond_df =city_data_df.loc[((city_data_df["Max Temp"]<=27.0) & (city_data_df["Max Temp"] >=21.0)) & (city_data_df["Wind Speed"]< 4.5) & (city_data_df["Cloudiness"]==0),:] 


# Drop any rows with null values
ideal_cond_df = ideal_cond_df.dropna()

# Display sample data
ideal_cond_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
94,94,pithoro,25.5167,69.3833,23.83,21,0,4.20,PK,1704542097
103,103,chabahar,25.2919,60.6430,23.53,57,0,3.82,IR,1704542104
180,180,san agustin de valle fertil,-30.6335,-67.4682,24.82,57,0,2.12,AR,1704542201


# Step 3: Create a new DataFrame called hotel_df.

In [36]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cond_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
94,pithoro,PK,25.5167,69.3833,21,
103,chabahar,IR,25.2919,60.6430,57,
180,san agustin de valle fertil,AR,-30.6335,-67.4682,57,


# Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [41]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apikey":geoapify_key,
    "limit":20,
    "categories":"accommodation.hotel"
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat= row["Lat"]
    lon= row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
pithoro - nearest hotel: No hotel found
chabahar - nearest hotel: No hotel found
san agustin de valle fertil - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
94,pithoro,PK,25.5167,69.3833,21,No hotel found
103,chabahar,IR,25.2919,60.6430,57,No hotel found
180,san agustin de valle fertil,AR,-30.6335,-67.4682,57,No hotel found


# Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [40]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=650,
    frame_height=500,
    size="Humidity",
    color="City",
    hover_cols=["Hotel Name","Country"]
    )

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)